<center><h1 style="color:#D4AF37"> ⚡⚡ AEMR Data Analysis ⚡⚡</h1>

<img src = "https://images.squarespace-cdn.com/content/v1/551972d8e4b0d571edc2e2c8/8f3abd33-0916-4415-9b24-593eea1d1e3c/Screen+Shot+2022-04-20+at+12.41.24+PM.png">

  


<h1 style="color:#D4AF37"> What's the Business Problem? 💰</h1>

The American Energy Market Regulator (AEMR) is responsible for looking after the
United States of America’s domestic energy network. The regulator’s responsibility is to
ensure that America’s energy network remains reliable with minimal disruptions, which
are known as outages. 

There are four key types of outages:

● Consequential

● Forced 

● Opportunistic 

● Planned 

Recently, the AEMR management team has been increasingly aware of a large number
of energy providers that submitted outages over the 2016 and 2017 calendar years. The
management team has expressed a desire to have the following two areas of concern
addressed:

<b> A) Energy Stability and Market Outages
    <p>
B) Energy Losses and Market Reliability </b>

As an analyst within the data and reporting team, you have been asked to address these
two immediate areas of concern. Feel free to also explore beyond the queries asked and provide additional insights that you feel may be of interest to the management team. 

<h3 style="color:#D4AF37">  SQLite Refresher ⚙️ </h3>



In [1]:
!pip install ipython-sql
!pip install prettytable==3.12

In [2]:
import requests
from IPython.core.magic import register_line_magic
from IPython.display import HTML
import sqlite3

@register_line_magic
def load_sqlite_db(url):
    response = requests.get(url)

    if response.status_code == 200:
        with open('temp_db_file.db', 'wb') as file:
            file.write(response.content)
        print('SQLite database file downloaded successfully.')
    else:
        print('Failed to download the SQLite database file.')

sqlite_db_url = 'https://raw.githubusercontent.com/chrishuisb1990/practice_datasets/main/AEMR.db'

%load_sqlite_db $sqlite_db_url

%load_ext sql

%sql sqlite:///temp_db_file.db

%config SqlMagic.style = '_DEPRECATED_DEFAULT'

SQLite database file downloaded successfully.


<h3 style="color:#D4AF37"> ⚡ Part I. Energy Stability & Market Outages ⚡ </h3>

Energy stability is one of the key themes the AEMR management team cares about. To ensure energy security and reliability, AEMR needs to understand the following:
<p>
<b>

<li> What are the most common outage types and how long do they tend to last? 
<li> How frequently do the outages occur? 
<li> Are there any energy providers which have more outages than their peers which may be indicative of being unreliable? 
    
<p>



<h3 style="color:#D4AF37"> Question One </h3> 
Do we notice anything regarding the trends for specific Outages over the 2016 / 2017 Period?

In [73]:
%%sql
SELECT COUNT(Status) AS Total_Number_Outages, Outage_Reason, Year 
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
Group By Outage_Reason, Year
Order By Year
LIMIT 5
;

 * sqlite:///temp_db_file.db
Done.


Total_Number_Outages,Outage_Reason,Year
181,Consequential,2016
1264,Forced,2016
106,Opportunistic Maintenance (Planned),2016
380,Scheduled (Planned),2016
127,Consequential,2017


Now how about examining some monthly trends? Do we note any behaviours across the months that indicate certain months having more reliability issues over other months? 


In [5]:
%%sql
SELECT COUNT(Status) AS Total_Number_Outages, Month,Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved' and Year =2016
Group By Month, Year	
Order By Month,Year	
LIMIT 5


 * sqlite:///temp_db_file.db
Done.


Total_Number_Outages,Month,Year
191,1,2016
227,2,2016
136,3,2016
134,4,2016
174,5,2016


Interesting. We've now identified a specific outage type is rapidly increasing from 2016 to 2017. However, frequency is just one aspect we care about. We also care about the <b> `duration` </b> of our ouatges.

In other words, if an outage is very short, we aren't as concerned. However, if the outage is very long, this then has the risk of threatening our energy supplies. We want to identify the problematic energy providers here. This leads us to our next question below. 

In [89]:
%%sql
SELECT
    Participant_Code,
    Outage_Reason,
    strftime('%Y', Start_Time) AS Year,
    COUNT(*) AS Total_Number_Outage_Events,
    ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))), 2) AS Average_Outage_Duration_In_Days
FROM AEMR_Outage_Table
WHERE 
    Status = 'Approved' AND
    strftime('%Y', Start_Time) IN ('2016', '2017')
GROUP BY 
    Participant_Code,
    Outage_Reason,
    Year
ORDER BY 
    Total_Number_Outage_Events DESC,
    Outage_Reason,
    Year

LIMIT 5;


 * sqlite:///temp_db_file.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days
AURICON,Forced,2017,490,0.07
GW,Forced,2016,317,0.38
GW,Forced,2017,227,1.06
AURICON,Forced,2016,208,0.07
AUXC,Forced,2016,206,0.08


Now we're getting somewhere...!
We've identified participants who are having many outages, as well as participants who have been offline for the longest durations.

Armed with this information, it's important we're able to classify our participants accordingly based on reliability metrics of uptime.

We classify a participant based off the following criteria:
<b>
<li> High Risk - On average, the participant is unavailable for > 24 Hours (1 Day)</li>
<li> Medium Risk - On average, the participant is unavailable between 12 and 24 Hours </li>
<li> Low Risk - On average, the participant is unavailable for less than 12 Hours</li> 
</b>

In [11]:
%%sql
SELECT
    Participant_Code,
    Outage_Reason,
    strftime('%Y', Start_Time) AS Year,
    COUNT(*) AS Total_Number_Outage_Events,
    ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))), 2) AS Average_Outage_Duration_In_Days,
    CASE
        WHEN AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))) > 1 THEN 'High Risk'
        WHEN AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))) BETWEEN 0.5 AND 1 THEN 'Medium Risk'
        ELSE 'Low Risk'
    END AS Risk_Classification
FROM AEMR_Outage_Table
WHERE 
    Status = 'Approved'
    AND Outage_Reason = 'Forced'
    AND strftime('%Y', Start_Time) IN ('2016', '2017')
GROUP BY 
    Participant_Code,
    Outage_Reason,
    Year
ORDER BY 
    Participant_Code,
    Year DESC
LIMIT 3;


 * sqlite:///temp_db_file.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days,Risk_Classification
AURICON,Forced,2017,490,0.07,Low Risk
AURICON,Forced,2016,208,0.07,Low Risk
AUXC,Forced,2017,120,0.02,Low Risk


<b>Now that we've classified our participants as either `High Risk`, `Medium Risk` or `Low Risk`, we want to dig a little deeper.

Does it make sense that `Consequential`, `Opportunistic` or `Planned` aren't considered regarding the Risk Category?

Perhaps we should refine our category accordingly by ensuring we focus our Risk Category on labelling only `Forced` Outages as being a Risk. After all, Forced Outages are the unplanned outages that risk the security of the electricity grid.

Let's add two additional criteria to our classification considering `Total Number of Outage Events` and `Outage Type`.

We've summarised these below:

<b>
<li> High Risk - On average, the participant is unavailable for > 24 Hours (1 Day) OR the Total Number of Outage Events > 20 </li>
<li> Medium Risk - On average, the participant is unavailable between 12 and 24 Hours OR the Total Number of Outage Events is Between 10 and 20 </li>
<li> Low Risk - On average, the participant is unavailable for less than 12 Hours OR the Total Number of Outage Events < 10 </li> 
<li> If Outage Type is not forced, then N/A
</b>


In [9]:
%%sql
SELECT
    Participant_Code,
    Outage_Reason,
    strftime('%Y', Start_Time) AS Year,
    COUNT(*) AS Total_Number_Outage_Events,
    ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))), 2) AS Average_Outage_Duration_In_Days,
    CASE 
        WHEN Outage_Reason = 'Forced' AND AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))) > 1 THEN 'High Risk'
        WHEN Outage_Reason = 'Forced' AND AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))) BETWEEN 0.5 AND 1 THEN 'Medium Risk'
        WHEN Outage_Reason = 'Forced' THEN 'Low Risk'
        ELSE 'N/A'
    END AS Risk_Classification
FROM AEMR_Outage_Table
WHERE 
    Status = 'Approved'
    AND strftime('%Y', Start_Time) IN ('2016', '2017')
GROUP BY 
    Participant_Code,
    Outage_Reason,
    Year
ORDER BY 
    Participant_Code,
    Year,
    Outage_Reason DESC




 * sqlite:///temp_db_file.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days,Risk_Classification
AURICON,Scheduled (Planned),2016,46,1.89,N/A
AURICON,Opportunistic Maintenance (Planned),2016,3,0.33,N/A
AURICON,Forced,2016,208,0.07,Low Risk
AURICON,Consequential,2016,41,0.13,N/A
AURICON,Scheduled (Planned),2017,45,1.45,N/A
AURICON,Forced,2017,490,0.07,Low Risk
AURICON,Consequential,2017,42,0.21,N/A
AUXC,Scheduled (Planned),2016,2,1.25,N/A
AUXC,Forced,2016,206,0.08,Low Risk
AUXC,Consequential,2016,1,0.96,N/A


<h3 style="color:#D4AF37"> ⚡ Part II. Energy Losses & Market Reliability ⚡ </h3>

When an energy provider provides energy to the market, they are making a commitment to the market and saying; “We will supply X amount of energy to the market under a contractual obligation.” However, in a situation where the outages are Forced, the energy provider intended to provide energy but are unable to provide energy and are forced offline. <b style="color:Maroon">If many energy providers are forced offline at the same time it could cause an energy security risk that AEMR needs to mitigate. </b>

To ensure this doesn’t happen, the AEMR is interested in exploring the following 

<li> Of the outage types in 2016 and 2017, what percent were Forced Outage(s)?
<li> What was the average duration for a forced outage during both 2016 and 2017? Have we seen an increase in the average duration of forced outages?
<li> Which energy providers tended to have the largest number of forced outages?
    
<p>


    


<b> The proportion of Forced Outages that have occurred over the 2016 - 2017 Period.
Do we observe any particular increases regarding any Outage Types over this period? </b>

In [138]:
#### %%sql
SELECT
 
    COUNT(*) AS Total_Number_Outages,
    SUM(CASE WHEN Outage_Reason = 'Forced' THEN 1 ELSE 0 END) AS Total_Number_Forced_Outage_Events,
    ROUND(
        CAST(SUM(CASE WHEN Outage_Reason = 'Forced' THEN 1 ELSE 0 END) AS FLOAT) 
        / COUNT(*) * 100, 
        2
    ) AS Pct_Outage_Forced
FROM AEMR_Outage_Table
WHERE 
    Status = 'Approved' 
GROUP BY 
    Year
ORDER BY 
    Year
    ;


 * sqlite:///temp_db_file.db
Done.


Total_Number_Outages,Total_Number_Forced_Outage_Events,Pct_Outage_Forced
1931,1264,65.46
2171,1622,74.71


<b>  Great. It's clear to see now that `Forced Outages` are problematic for us. Not only are they the only outage type that generates financial losses as the Outage is unplanned, it seems there is a number of Energy Participants who have been having a significantly high number of Outages.

Now what can we do about this? 

Let's break our analysis down into Macro and Micro Analysis.
The total gives us the Overall Duration a participant is offline / has lost energy, however, it doesn't tell us how *frequently* this occurs. In other words, if we have one or two very big outages, it might contribute to very large totals.

However, perhaps an <b> average </b> can help us identify how big these Outages might really be, spread across the year!



<b> The `Total Number of Outages`, `Total Duration In Days`and `Total Energy Lost` of all valid `Outages` for each `participant code` and `facility_code`, sorted by `Total Energy Lost` in descending order and Ordered by the YEAR Category. 
    

In [11]:
%%sql
SELECT
    COUNT(*) AS Total_Number_Outages,
    ROUND(SUM(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))), 2) AS Total_Duration_In_Days,
    ROUND(SUM(Energy_Lost_MW), 2) AS Total_Energy_Lost,
    Outage_Reason,
    Participant_Code,
    Facility_Code,
    strftime('%Y', Start_Time) AS Year
FROM AEMR_Outage_Table
WHERE 
    Status = 'Approved'
    AND strftime('%Y', Start_Time) IN ('2016', '2017') 
AND Outage_Reason IN ('Scheduled (Planned)', 'Forced')
GROUP BY 
    Outage_Reason,
    Participant_Code,
    Facility_Code,
    Year
HAVING COUNT(*) <=227
ORDER BY 
    Year,
    Total_Energy_Lost DESC


   



 * sqlite:///temp_db_file.db
Done.


Total_Number_Outages,Total_Duration_In_Days,Total_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
157,129.6,13771.07,Forced,MELK,MELK_G7,2016
208,15.06,10696.28,Forced,AURICON,AURICON_PNJ_U1,2016
85,392.25,9668.79,Scheduled (Planned),MELK,MELK_G7,2016
69,34.06,9093.08,Forced,PMC,PMC_AG,2016
46,87.02,6964.8,Scheduled (Planned),AURICON,AURICON_PNJ_U1,2016
45,199.4,6450.0,Scheduled (Planned),GW,BW1_GREENWATERS_G2,2016
81,98.79,5881.52,Forced,PJRH,PJRH_GT11,2016
27,51.4,4839.0,Scheduled (Planned),PMC,PMC_AG,2016
29,32.33,4320.86,Forced,COLLGAR,COLLGAR_WF1,2016
53,20.0,4040.32,Forced,KORL,KORL_GT3,2016


<b> The `Average Duration In Days`and `Average Energy Lost` of all valid `FORCED OUTAGES` for each `participant code` and `facility_code` sorted by `Average Energy Lost` in descending order and Ordered by the YEAR Category. 


In [13]:
%%sql
SELECT
    ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))), 2) AS Avg_Duration_In_Days,
    ROUND(AVG(Energy_Lost_MW), 2) AS Avg_Energy_Lost,
    Outage_Reason,
    Participant_Code,
    Facility_Code,
    strftime('%Y', Start_Time) AS Year
FROM AEMR_Outage_Table
WHERE
    Status = 'Approved'
    AND Outage_Reason = 'Forced'
    AND strftime('%Y', Start_Time) IN ('2016', '2017')
GROUP BY
    Participant_Code,
    Facility_Code,
    Outage_Reason,
    Year
ORDER BY
    Year ASC,
    Avg_Energy_Lost DESC







 * sqlite:///temp_db_file.db
Done.


Avg_Duration_In_Days,Avg_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
1.11,149.0,Forced,COLLGAR,COLLGAR_WF1,2016
0.49,131.78,Forced,PMC,PMC_AG,2016
0.83,87.71,Forced,MELK,MELK_G7,2016
0.25,80.0,Forced,TSLA_MGT,TESLA_PICTON_G1,2016
0.38,76.23,Forced,KORL,KORL_GT3,2016
1.22,72.61,Forced,PJRH,PJRH_GT11,2016
2.24,56.32,Forced,ENRG,ENRG_KALGOORLIE_GT3,2016
0.79,55.0,Forced,MCG,MWF_MUMBIDA_WF1,2016
0.07,51.42,Forced,AURICON,AURICON_PNJ_U1,2016
0.38,49.69,Forced,GW,BW1_GREENWATERS_G2,2016



Which participants have contributed the most to the Energy Lost due to Forced Outages?


In [15]:
%%sql
WITH Forced_Energy AS (
    SELECT
        ROUND(AVG(Energy_Lost_MW), 2) AS Avg_Energy_Lost,
        ROUND(SUM(Energy_Lost_MW), 2) AS Total_Energy_Lost,
        Outage_Reason,
        Participant_Code,
        Facility_Code,
        strftime('%Y', Start_Time) AS Year
    FROM AEMR_Outage_Table
    WHERE 
        Status = 'Approved'
        AND Outage_Reason = 'Forced'
        AND strftime('%Y', Start_Time) IN ('2016', '2017')
    GROUP BY 
        Participant_Code, Facility_Code, Outage_Reason, Year
),
Total_By_Facility_Year AS (
    SELECT
        Facility_Code,
        strftime('%Y', Start_Time) AS Year,
        SUM(Energy_Lost_MW) AS Total_Loss_Per_Year
    FROM AEMR_Outage_Table
    WHERE 
        Status = 'Approved'
        AND strftime('%Y', Start_Time) IN ('2016', '2017')
    GROUP BY Facility_Code, Year
)

SELECT
    f.Avg_Energy_Lost,
    f.Total_Energy_Lost,
    ROUND((f.Total_Energy_Lost * 100.0) / t.Total_Loss_Per_Year, 2) AS Pct_Energy_Loss,
    f.Outage_Reason,
    f.Participant_Code,
    f.Facility_Code,
    f.Year
FROM Forced_Energy f
JOIN Total_By_Facility_Year t
  ON f.Facility_Code = t.Facility_Code AND f.Year = t.Year
ORDER BY f.Total_Energy_Lost DESC
;


 * sqlite:///temp_db_file.db
Done.


Avg_Energy_Lost,Total_Energy_Lost,Pct_Energy_Loss,Outage_Reason,Participant_Code,Facility_Code,Year
44.16,21639.55,71.81,Forced,AURICON,AURICON_PNJ_U1,2017
85.14,19326.56,78.4,Forced,GW,BW1_GREENWATERS_G2,2017
49.69,15751.38,60.62,Forced,GW,BW1_GREENWATERS_G2,2016
87.71,13771.07,49.46,Forced,MELK,MELK_G7,2016
51.42,10696.28,48.38,Forced,AURICON,AURICON_PNJ_U1,2016
58.11,10285.4,53.78,Forced,MELK,MELK_G7,2017
131.78,9093.08,60.99,Forced,PMC,PMC_AG,2016
72.61,5881.52,55.99,Forced,PJRH,PJRH_GT11,2016
141.21,5648.44,60.77,Forced,PMC,PMC_AG,2017
29.17,5016.67,96.75,Forced,TRMOS,TIWEST_COG1,2017


<b>Having identified the top 3 participants by Total Energy Loss being `GW`, `MELK` and `Auricon`; Write a SQL Statement calculating the `Total_Energy_Lost` each of these three `Participant_Codes` and the `Facility_Code`. Additionally, identify the `Description_Of_Outage` associated with the highest `Total_Energy_Lost` for each of the `Participant_Codes` and `Facility_Code` for each of the three participants. 
    
<u>Lastly,  the percentage of Energy Loss, attributed to these reasons!</u> </b>




In [162]:
%%sql
WITH Energy_Loss_Details AS (
    SELECT
        Participant_Code,
        Facility_Code,
        Description_Of_Outage,
        ROUND(SUM(Energy_Lost_MW), 2) AS Total_Energy_Lost,
        ROUND(
            (SUM(Energy_Lost_MW) * 100.0) 
            / SUM(SUM(Energy_Lost_MW)) OVER (PARTITION BY Participant_Code, Facility_Code), 
            2
        ) AS Pct_Energy_Loss,
        ROW_NUMBER() OVER (
            PARTITION BY Participant_Code, Facility_Code
            ORDER BY SUM(Energy_Lost_MW) DESC
        ) AS rank
    FROM AEMR_Outage_Table
    WHERE 
        Status = 'Approved'
        AND strftime('%Y', Start_Time) IN ('2016', '2017')
        AND Participant_Code IN ('GW', 'MELK', 'AURICON')
    GROUP BY
        Participant_Code,
        Facility_Code,
        Description_Of_Outage
)

SELECT
    Participant_Code,
    Facility_Code,
    Description_Of_Outage,
    Total_Energy_Lost,
    Pct_Energy_Loss,
    rank
FROM Energy_Loss_Details
WHERE rank = 1
ORDER BY Total_Energy_Lost DESC;



 * sqlite:///temp_db_file.db
Done.


Participant_Code,Facility_Code,Description_Of_Outage,Total_Energy_Lost,Pct_Energy_Loss,rank
GW,BW1_GREENWATERS_G2,Operational Issues caused real time forced outage.,28687.54,56.65,1
AURICON,AURICON_PNJ_U1,Full unit trip,6033.87,11.55,1
MELK,MELK_G7,Safety Issues,1100.0,2.34,1
